In [59]:
! pip install  -q -U langchain langchain-community  sentence-transformers faiss-cpu tiktoken langchain-google-genai

In [79]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.embeddings.sentence_transformer import HuggingFaceEmbeddings,SentenceTransformerEmbeddings # We can use both for embeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import RetrievalQA

# Load data , split chunk , store vectors


In [80]:
health_benifits_urls=["https://www.healthline.com/health/benefits-of-swimming","https://www.healthline.com/nutrition/13-benefits-of-yoga","https://www.healthline.com/health/fitness-exercise/running-everyday"]

def load_document(url):
  loader=WebBaseLoader(url)
  document=loader.load()
  return document

def document_splitter(document):
  splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
  document_split=splitter.split_documents(document)
  return document_split

def store_vector(document_split,model_id):
  embeddings=SentenceTransformerEmbeddings(model_name=model_id)
  vector_db=FAISS.from_documents(document_split,embeddings)
  vector_db.save_local("DB")
  return vector_db,True


# Chain

In [86]:
def prompt_format():
  template = """/
  Your are the expert of providing benifits of health,swimming etc.. if you dont know the answer just say dont know dont make unnecessary answers

  Context={context}
  Question: {question}

  Answer:
  """
  prompt=PromptTemplate(
      input_variables=['context','question'],
      template=template
  )

  return prompt

def retrieval_QA(vector_db,prompt):
  chain=RetrievalQA.from_chain_type(
    llm=GoogleGenerativeAI(model="models/text-bison-001",google_api_key="your key"),
    retriever=vector_db.as_retriever(kwargs={"k":3}),
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True)
  return chain

# Function call

In [83]:
model_id="all-MiniLM-L6-v2" # Embedding repo model id

document=load_document(health_benifits_urls)
splitted_docs=document_splitter(document)
vector_db,status=store_vector(splitted_docs,model_id)

if status:
  print("Stored Vector Successfully")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Stored Vector Successfully


In [87]:
prompt=prompt_format()
ques="Explain Swimming safety?"
qa_chain=retrieval_QA(vector_db,prompt)
response=qa_chain({"query":ques})

In [89]:
response['result']

'The following swim safety tips can help reduce your risk from swimming: Swim in areas that are designated for swimming, like pools and roped off sections of lakes and other bodies of water. If possible, swim in areas that are supervised by lifeguards.If you aren’t swimming with lifeguard supervision, bring a buddy.Consider taking swimming lessons if you’re new to the sport. You can enroll in age-appropriate classes through the Red Cross and through other programs in your area.Swimming outdoors? Wear sunscreen of at least SPF 15 or higher to protect your skin. You may also want to avoid swimming between the hours of 10 a.m. and 4 p.m., when the sun is at its strongest.'

In [75]:
print(response['result'])

1. Swim in areas that are designated for swimming, like pools and roped off sections of lakes and other bodies of water.
2. If possible, swim in areas that are supervised by lifeguards.
3. If you aren’t swimming with lifeguard supervision, bring a buddy.
4. Consider taking swimming lessons if you’re new to the sport.
5. Swimming outdoors? Wear sunscreen of at least SPF 15 or higher to protect your skin.
6. You may also want to avoid swimming between the hours of 10 a.m. and 4 p.m., when the sun’s UV rays are strongest.


# Retrivel data from vector DB

In [93]:
generate=True
while generate:
  question=input("Ask your questions?..")
  if question =="exit":
    generate=False
  else:
    prompt=prompt_format()
    qa_chain=retrieval_QA(vector_db,prompt)
    response=qa_chain({"query":question})
    print(response['result'])


Ask your questions?..Swim lessons what its?
offered in private or group settings. In lessons, you’ll learn different strokes, breathing techniques, and other handy tips for getting the most from your workout.
Ask your questions?..exit
